In [1]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from IPython.display import display, Markdown
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, DateType
from pyspark.sql.functions import when, count, col, countDistinct, \
                                    desc, asc, round, date_format, \
                                    concat_ws, expr, month, \
                                    first, lit, max, min, stddev, avg

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
